In [1]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [2]:
sys.path.append('..')

In [3]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

<module 'modules.Elastic_Module' from '/home/francisco/Desktop/repos/opendata/ETL/../modules/Elastic_Module.py'>

In [4]:
from modules.Elastic_Module import *

In [10]:
url_ckan='https://datos.gob.cl/api/3/action/package_list'
url_package='https://datos.gob.cl/api/3/action/package_show?id='
url_package_search='https://datos.gob.cl/api/3/action/package_search?'

In [11]:
#Headers user agent simular browser para no obtener error 403 (prohibido)
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
}

In [12]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan,headers=headers) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [14]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start,headers=headers)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [16]:
recursos=[]
for p in packs:
    tags=[]
    for t in p['tags']:
        tags.append(t['name'])
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        d['tags']=tags

        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df      

In [17]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato','name':'resource_title',
                                 }) #cambiar nombres de columnas

In [18]:
recursos=recursos.drop(['mimetype','cache_url','cache_last_updated','hash','url_type',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

In [19]:
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [20]:
data=recursos.to_dict(orient='records')

In [26]:
#guardar paso en csv (server)
#path='../ETLcache/'
#now = dt.now()
#date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [26]:
#guardar paso en csv (local)
path='../../data/backup_opendata/backup_opendata_2_12_21'
#now = dt.now()
date_time = ''

In [27]:
recursos.to_csv(path+date_time+'.csv')

In [36]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria (server)
#with open(path+date_time+'.pickle', 'wb') as f:
#    pickle.dump(packs, f)
#del(packs)

In [28]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria (server)
with open(path+date_time+'.pickle', 'wb') as f:
    pickle.dump(packs, f)
del(packs)

In [13]:
#es = Elasticsearch(hosts = [ES_HOST],http_auth=('elastic', '                    # replace with the empty string'))

In [31]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=#os.environ['ELASTIC_PASSWD']
args['DOMAIN_ELASTIC']=#os.environ['DOMAIN_ELASTIC']
args['ELASTIC_PORT']=9200#os.environ['ELASTIC_PORT']

In [32]:
ES_HOST = "https://"+args['DOMAIN_ELASTIC']
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [33]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [34]:
es.ping()

True


In [35]:
#generar campos concatenados (parámetros)
generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
                               'head':20, #numero de caracteres a dejar
                                'sep':'_' #separador entre campos
                               }

In [36]:
es.generateID_params_custom=generateID_params_custom

In [37]:
es.add_docs(recursos)

In [38]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [39]:
indexes=[
    ('ckan','ckan')
        ]

In [40]:
#generar bulk data
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1796875 Kbs
N° docs: 10445.0, N° chunks: 14


In [41]:
len(es.chunks[0])

1500

In [42]:
es.clear_index(i[0])

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [43]:
es.bulk_index()

bulk indexing...:index---> ckan
 indexing chunk : 13 of 14finished bulk indexing
No recorded ES indexing errors
No recorded ES indexing exceptions


In [44]:
es.errors

AttributeError: 'elastic_connection' object has no attribute 'errors'

In [45]:
es.test_index(i[0],n=15)

/home/francisco/Desktop/repos/opendata/ETL/../modules/Elastic_Module.py:78: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize([x['_source'] for x in res['hits']['hits']])


,package_id,datastore_contains_all_records_of_source_file,datastore_active,size,original_url,resource_id,set_url_type,state,ignore_hash,description,...,tags,inst.,inst_name,inst_id,document_image_upload,document_clear_upload,document_image_url,dia,year,mes
0,3f0627f9-1d2f-4763-ab1c-2ba875893361,True,True,353076.0,https://datos.gob.cl/dataset/3f0627f9-1d2f-476...,0010ba8d-ca21-4c3a-8c79-551b64b8f2b3,False,active,True,Información de Ejecución Presupuestaria a Octu...,...,"[2020, dipres, ejecución, octubre, presupuesta...",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
1,3f0627f9-1d2f-4763-ab1c-2ba875893361,True,True,8735024.0,https://datos.gob.cl/dataset/3f0627f9-1d2f-476...,71607249-5b53-4d93-a2e4-90ca8ab7bf5a,False,active,True,Información de Ejecución Presupuestaria a Octu...,...,"[2020, dipres, ejecución, octubre, presupuesta...",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
2,3f0627f9-1d2f-4763-ab1c-2ba875893361,True,True,1646.0,https://datos.gob.cl/dataset/3f0627f9-1d2f-476...,35a0e991-90c3-40b9-afce-7c95920c4e59,False,active,True,Diccionario de Datos Abiertos de Ejecución Pre...,...,"[2020, dipres, ejecución, octubre, presupuesta...",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
3,e64029a8-f4d4-4965-8968-709929c46beb,True,True,3271.0,https://datos.gob.cl/dataset/e64029a8-f4d4-496...,bcc4696c-5171-40c5-a38b-c88bb28e3df7,False,active,True,Información de Ley de Presupuestos Inicial y V...,...,"[2020, dipres, ley, octubre, presupuesto]",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
4,e64029a8-f4d4-4965-8968-709929c46beb,True,True,58727.0,https://datos.gob.cl/dataset/e64029a8-f4d4-496...,43c43cd6-7845-414f-8611-0868d7dc3753,False,active,True,Información de Ley de Presupuestos Inicial y V...,...,"[2020, dipres, ley, octubre, presupuesto]",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
5,e64029a8-f4d4-4965-8968-709929c46beb,True,True,417534.0,https://datos.gob.cl/dataset/e64029a8-f4d4-496...,0de789a9-6158-467b-a602-4cc282a1cd5b,False,active,True,Información de Ley de Presupuestos Inicial y V...,...,"[2020, dipres, ley, octubre, presupuesto]",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
6,e64029a8-f4d4-4965-8968-709929c46beb,True,True,2777123.0,https://datos.gob.cl/dataset/e64029a8-f4d4-496...,9c5391de-53a2-4ef9-8849-eb2728eca53d,False,active,True,Información de Ley de Presupuestos Inicial y V...,...,"[2020, dipres, ley, octubre, presupuesto]",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
7,e64029a8-f4d4-4965-8968-709929c46beb,True,True,1642.0,https://datos.gob.cl/dataset/e64029a8-f4d4-496...,c3802150-1bfe-46da-b4a1-fa76b3371415,False,active,True,Diccionario de Datos Abiertos de Ley de Presup...,...,"[2020, dipres, ley, octubre, presupuesto]",Dirección de Presupuestos,direccion_de_presupuestos,AE002,None,None,None,2020-12-04,2020.0,12.0
8,f1918e3c-fef8-4d28-995f-ddbfce352565,True,True,1055744.0,https://datos.gob.cl/dataset/f1918e3c-fef8-4d2...,e53591c7-b465-4809-a7ac-ca083ca1063a,False,active,False,Patentes Comerciales Segundo Semestre Año 2020,...,[Patentes Comerciales Segundo Semestre Año 2020],Municipalidad de San Felipe,municipalidad_de_san_felipe,MU286,None,None,None,2020-12-03,2020.0,12.0
9,47048cfa-7449-4a56-9f05-10bd88b6f8ec,True,True,NaN,https://prod.datos.gob.cl/dataset/47048cfa-744...,ad286aeb-1e2e-4b99-b9a2-3bf1ec011f32,False,active,True,Numero de Consultas mensual para las Unidades ...,...,"[atención de urgencias, emergencias, urgencias]",Hospital Gustavo Fricke,hospital_gustavo_fricke,AO098,None,None,None,2020-11-03,2020.0,11.0


In [46]:
es.list_indices()

['.security-7',
 'kibana_sample_data_flights',
 '.kibana_task_manager',
 '.kibana_1',
 'ckan']